In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pyvista as pv
import CollisionModule

In [2]:
import torch
from scipy import interpolate

In [3]:
DXsec_Al = np.load('./DXsec_Al_1e8.npy')

In [4]:
import DSMC_depoSimulator_generateWithtime

In [143]:
# cell model construct

#              - end
#            -
#          -
#        -
#      -
#    - start

# direction x, y; start (x, y, z)
def slide2D_fractionZ(film, start, end, direction, value):
    if direction == 'y':
        slit = np.linspace(0, np.abs(end[2] - start[2]), np.abs(end[1] - start[1]))
        print('y', slit)
        for i in range(np.abs(end[1] - start[1])):
            if end[1] > start[1]:
                film[start[0]:end[0], start[1] + i, start[2]:start[2] + int(slit[i])] = value
                for j in range(np.abs(int(slit[0]-slit[1]))):
                    film[start[0]:end[0], start[1] + i,start[2]+int(slit[i])+j] = 1/(value+1)*(slit[0]-slit[1]-j)
            elif end[1] < start[1]:
                film[start[0]:end[0], start[1] - i, start[2]:start[2] + int(slit[i])] = value
                for j in range(np.abs(int(slit[0]-slit[1]))):
                    film[start[0]:end[0], start[1] - i,start[2]+int(slit[i])+j] = 1/(value+1)*(slit[0]-slit[1]-j)
    elif direction == 'x':
        slit = np.linspace(0, np.abs(end[2] - start[2]), np.abs(end[0] - start[0]))
        print('x',slit)
        for i in range(np.abs(end[0] - start[0])):
            print(i)
            if end[0] > start[0]:
                film[start[0] + i, start[1]:end[1], start[2]:start[2] + int(slit[i])] = value
                for j in range(np.abs(int(slit[0]-slit[1]))):
                    film[start[0] + i, start[1]:end[1], start[2]+int(slit[i])+j] = 1/(value+1)*(slit[0]-slit[1]-j)
            elif end[0] < start[0]:
                film[start[0] - i, start[1]:end[1], start[2]:start[2] + int(slit[i])] = value
                for j in range(np.abs(int(slit[0]-slit[1]))):
                    film[start[0] - i, start[1]:end[1], start[2]+int(slit[i])+j] = 1/(value+1)*(slit[0]-slit[1]-j)
    return film

In [52]:
45.2/38

1.1894736842105265

In [75]:
115/96

1.1979166666666667

In [66]:
2.46/45.2

0.05442477876106194

In [68]:
6/115

0.05217391304347826

In [78]:
13.5/38

0.35526315789473684

In [79]:
34/96

0.3541666666666667

In [82]:
(45.2-25.3)/45.2

0.44026548672566373

In [91]:
(115-65)/115

0.43478260869565216

In [90]:
115-64

51

In [94]:
34/38

0.8947368421052632

In [97]:
86/96

0.8958333333333334

In [100]:
108/34

3.176470588235294

In [107]:
273/86

3.1744186046511627

In [111]:
(108-60.7869)/108

0.4371583333333333

In [120]:
(273-154)/273

0.4358974358974359

In [122]:
(45.2-19.1)/45.2

0.577433628318584

In [128]:
(115-49)/115

0.5739130434782609

In [129]:
115-49

66

In [251]:
# cell model construct

#              - end
#            -
#          -
#        -
#      -
#    - start

# direction x, y; start (x, y, z)
def slide2D_fractionX(film, start, end, direction, fraction, value):
    if fraction == '+':
        if direction == 'y':
            slit = np.linspace(0, np.abs(end[0] - start[0]), np.abs(end[1] - start[1]))
            fraction = np.abs(int(slit[0]-slit[1]))
            print('y', slit)
            print('fraction', fraction)
            for i in range(np.abs(end[1] - start[1])):
                if end[1] > start[1]:
                    film[start[0]:start[0] + int(slit[i]), start[1] + i, start[2]:end[2]] = value
                    for j in range(fraction):
                        film[start[0]+int(slit[i])+j, start[1] + i,start[2]:end[2]] = 1/(fraction+1)*(fraction-j)
                elif end[1] < start[1]:
                    film[start[0]:start[0] + int(slit[i]), start[1] - i, start[2]:end[2]] = value
                    for j in range(fraction):
                        film[start[0]+int(slit[i])+j, start[1] - i,start[2]:end[2]] = 1/(fraction+1)*(fraction-j)
        elif direction == 'z':
            slit = np.linspace(0, np.abs(end[0] - start[0]), np.abs(end[2] - start[2]))
            fraction = np.abs(int(slit[0]-slit[1]))
            print('z', slit)
            print('fraction', fraction)
            for i in range(np.abs(end[2] - start[2])):
                if end[2] > start[2]:
                    film[start[0]:start[0] + int(slit[i]), start[1]:end[1], start[2] + i] = value
                    for j in range(fraction):
                        film[start[0]+int(slit[i])+j, start[1]:end[1], start[2] + i] = 1/(fraction+1)*(fraction-j)
                elif end[2] < start[2]:
                    film[start[0]:start[0] + int(slit[i]), start[1]:end[1], start[2] - i] = value
                    for j in range(fraction):
                        film[start[0]+int(slit[i])+j, start[1]:end[1], start[2] - i] = 1/(fraction+1)*(fraction-j)
    elif fraction == '-':
        if direction == 'y':
            slit = np.linspace(0, np.abs(end[0] - start[0]), np.abs(end[1] - start[1]))
            fraction = np.abs(int(slit[0]-slit[1]))
            print('y', slit)
            print('fraction', fraction)
            for i in range(np.abs(end[1] - start[1])):
                if end[1] > start[1]:
                    film[start[0] - int(slit[i]):start[0], start[1] + i, start[2]:end[2]] = value
                    for j in range(fraction):
                        film[start[0]-int(slit[i])-j, start[1] + i,start[2]:end[2]] = 1/(fraction+1)*(fraction-j)
                elif end[1] < start[1]:
                    film[start[0] - int(slit[i]):start[0], start[1] - i, start[2]:end[2]] = value
                    for j in range(fraction):
                        film[start[0]-int(slit[i])-j, start[1] - i,start[2]:end[2]] = 1/(fraction+1)*(fraction-j)
        elif direction == 'z':
            slit = np.linspace(0, np.abs(end[0] - start[0]), np.abs(end[2] - start[2]))
            fraction = np.abs(int(slit[0]-slit[1]))
            print('z', slit)
            print('fraction', fraction)
            for i in range(np.abs(end[2] - start[2])):
                if end[2] > start[2]:
                    film[start[0] - int(slit[-i]):start[0], start[1]:end[1], start[2] + i] = value
                    for j in range(fraction):
                        film[start[0]-int(slit[-i])-j, start[1]:end[1], start[2] + i] = 1/(fraction+1)*(fraction-j)
                elif end[2] < start[2]:
                    film[start[0] - int(slit[-i]):start[0], start[1]:end[1], start[2] - i] = value
                    for j in range(fraction):
                        film[start[0]-int(slit[-i])-j, start[1]:end[1], start[2] - i] = 1/(fraction+1)*(fraction-j)
    return film

In [255]:
def slide2D_fractionZ(film, start, end, direction, fraction, value):
    if fraction == '+':
        if direction == 'y':
            slit = np.linspace(0, np.abs(end[2] - start[2]), np.abs(end[1] - start[1]))
            fraction = np.abs(int(slit[0]-slit[1]))
            print('y', slit)
            print('fraction', fraction)
            for i in range(np.abs(end[1] - start[1])):
                if end[1] > start[1]:
                    film[start[0]:end[0], start[1] + i, start[2]:start[2] + int(slit[i])] = value
                    for j in range(fraction):
                        film[start[0]:end[0], start[1] + i,start[2]+int(slit[i])+j] = 1/(fraction+1)*(fraction-j)
                elif end[1] < start[1]:
                    film[start[0]:end[0], start[1] - i, start[2]:start[2] + int(slit[i])] = value
                    for j in range(fraction):
                        film[start[0]:end[0], start[1] - i,start[2]+int(slit[i])+j] = 1/(fraction+1)*(fraction-j)
        elif direction == 'x':
            slit = np.linspace(0, np.abs(end[2] - start[2]), np.abs(end[0] - start[0]))
            fraction = np.abs(int(slit[0]-slit[1]))
            print('x', slit)
            print('fraction', fraction)
            for i in range(np.abs(end[2] - start[2])):
                if end[0] > start[0]:
                    film[start[0] + i, start[1]:end[1], start[2]:start[2] + int(slit[i])] = value
                    for j in range(fraction):
                        film[start[0] + i, start[1]:end[1], start[2]+int(slit[i])+j] = 1/(fraction+1)*(fraction-j)
                elif end[0] < start[0]:
                    film[start[0] - i, start[1]:end[1], start[2]:start[2] + int(slit[i])] = value
                    for j in range(fraction):
                        film[start[0] - i, start[1]:end[1], start[2]+int(slit[i])+j] = 1/(fraction+1)*(fraction-j)
    elif fraction == '-':
        if direction == 'y':
            slit = np.linspace(0, np.abs(end[2] - start[2]), np.abs(end[1] - start[1]))
            fraction = np.abs(int(slit[0]-slit[1]))
            print('y', slit)
            print('fraction', fraction)
            for i in range(np.abs(end[1] - start[1])):
                if end[1] > start[1]:
                    film[start[0]:end[0], start[1] + i, start[2] - int(slit[i]):start[2]] = value
                    for j in range(fraction):
                        film[start[0]:end[0], start[1] + i,start[2]-int(slit[i])-j] = 1/(fraction+1)*(fraction-j)
                elif end[1] < start[1]:
                    film[start[0]:end[0], start[1] - i, start[2] - int(slit[i]):start[2]] = value
                    for j in range(fraction):
                        film[start[0]:end[0], start[1] - i,start[2]-int(slit[i])-j] = 1/(fraction+1)*(fraction-j)
        elif direction == 'x':
            slit = np.linspace(0, np.abs(end[2] - start[2]), np.abs(end[0] - start[0]))
            fraction = np.abs(int(slit[0]-slit[1]))
            print('x', slit)
            print('fraction', fraction)
            for i in range(np.abs(end[2] - start[2])):
                if end[0] > start[0]:
                    film[start[0] + i, start[1]:end[1], start[2] - int(slit[i]):start[2]] = value
                    for j in range(fraction):
                        film[start[0] + i, start[1]:end[1], start[2] - int(slit[i]):start[2]] = 1/(fraction+1)*(fraction-j)
                elif end[0] < start[0]:
                    film[start[0] - i, start[1]:end[1], start[2] - int(slit[i]):start[2]] = value
                    for j in range(fraction):
                        film[start[0] - i, start[1]:end[1], start[2] - int(slit[i]):start[2]] = 1/(fraction+1)*(fraction-j)
    return film

In [504]:
def slide3D_fractionZ(film, start, end, direction, fraction, value):
    if fraction == '+':
        if direction == 'y':
            slit = np.linspace(0, np.abs(end[2] - start[2]), np.abs(end[1] - start[1]))
            fraction = np.abs(int(slit[0]-slit[1]))
            # print('y', slit)
            # print('fraction', fraction)
            for i in range(np.abs(end[1] - start[1])):
                if end[1] > start[1]:
                    film[start[0], start[1] + i, start[2]:start[2] + int(slit[i])] = value
                    for j in range(fraction):
                        film[start[0], start[1] + i,start[2]+int(slit[i])+j] = 1/(fraction+1)*(fraction-j)
                elif end[1] < start[1]:
                    film[start[0], start[1] - i, start[2]:start[2] + int(slit[i])] = value
                    for j in range(fraction):
                        film[start[0], start[1] - i,start[2]+int(slit[i])+j] = 1/(fraction+1)*(fraction-j)
        elif direction == 'x':
            slit = np.linspace(0, np.abs(end[2] - start[2]), np.abs(end[0] - start[0]))
            fraction = np.abs(int(slit[0]-slit[1]))
            # print('x', slit)
            # print('fraction', fraction)
            for i in range(np.abs(end[2] - start[2])):
                if end[0] > start[0]:
                    film[start[0] + i, start[1]:end[1], start[2]:start[2] + int(slit[i])] = value
                    for j in range(fraction):
                        film[start[0] + i, start[1]:end[1], start[2]+int(slit[i])+j] = 1/(fraction+1)*(fraction-j)
                elif end[0] < start[0]:
                    film[start[0] - i, start[1]:end[1], start[2]:start[2] + int(slit[i])] = value
                    for j in range(fraction):
                        film[start[0] - i, start[1]:end[1], start[2]+int(slit[i])+j] = 1/(fraction+1)*(fraction-j)
    elif fraction == '-':
        if direction == 'y':
            slit = np.linspace(0, np.abs(end[2] - start[2]), np.abs(end[1] - start[1]))
            fraction = np.abs(int(slit[0]-slit[1]))
            print('y', slit)
            print('fraction', fraction)
            for i in range(np.abs(end[1] - start[1])):
                if end[1] > start[1]:
                    film[start[0]:end[0], start[1] + i, start[2] - int(slit[i]):start[2]] = value
                    for j in range(fraction):
                        film[start[0]:end[0], start[1] + i,start[2]-int(slit[i])-j] = 1/(fraction+1)*(fraction-j)
                elif end[1] < start[1]:
                    film[start[0]:end[0], start[1] - i, start[2] - int(slit[i]):start[2]] = value
                    for j in range(fraction):
                        film[start[0]:end[0], start[1] - i,start[2]-int(slit[i])-j] = 1/(fraction+1)*(fraction-j)
        elif direction == 'x':
            slit = np.linspace(0, np.abs(end[2] - start[2]), np.abs(end[0] - start[0]))
            fraction = np.abs(int(slit[0]-slit[1]))
            print('x', slit)
            print('fraction', fraction)
            for i in range(np.abs(end[2] - start[2])):
                if end[0] > start[0]:
                    film[start[0] + i, start[1]:end[1], start[2] - int(slit[i]):start[2]] = value
                    for j in range(fraction):
                        film[start[0] + i, start[1]:end[1], start[2] - int(slit[i]):start[2]] = 1/(fraction+1)*(fraction-j)
                elif end[0] < start[0]:
                    film[start[0] - i, start[1]:end[1], start[2] - int(slit[i]):start[2]] = value
                    for j in range(fraction):
                        film[start[0] - i, start[1]:end[1], start[2] - int(slit[i]):start[2]] = 1/(fraction+1)*(fraction-j)
    return film

In [241]:
film = np.zeros((277, 100, 120))

bottom = 5
film[:, :, 0:bottom] = 10 # bottom

side_wall = 2
film[:, :side_wall, :] = 10 # left
film[:, -side_wall:, :] = 10 # right


# #dfsdfsdfasdf
# height = 80
# left_side = 75
# right_side = 75

# film = slide2D_fractionZ(film=film, start=[2, 67, 5], end=[88, 98, 56], direction='y', value=10)
# film = slide2D_fractionZ(film=film, start=[2, 32, 5], end=[88, 1, 56], direction='y', value=10)

film = slide2D_fractionX(film=film, start=[275, 2, 4], end=[88, 98, 66], direction='z', fraction='-', value=10)

z [  0.           3.06557377   6.13114754   9.19672131  12.26229508
  15.32786885  18.39344262  21.45901639  24.52459016  27.59016393
  30.6557377   33.72131148  36.78688525  39.85245902  42.91803279
  45.98360656  49.04918033  52.1147541   55.18032787  58.24590164
  61.31147541  64.37704918  67.44262295  70.50819672  73.57377049
  76.63934426  79.70491803  82.7704918   85.83606557  88.90163934
  91.96721311  95.03278689  98.09836066 101.16393443 104.2295082
 107.29508197 110.36065574 113.42622951 116.49180328 119.55737705
 122.62295082 125.68852459 128.75409836 131.81967213 134.8852459
 137.95081967 141.01639344 144.08196721 147.14754098 150.21311475
 153.27868852 156.3442623  159.40983607 162.47540984 165.54098361
 168.60655738 171.67213115 174.73770492 177.80327869 180.86885246
 183.93442623 187.        ]


In [252]:
film = np.zeros((40, 20, 20))

film = slide2D_fractionX(film=film, start=[0, 0, 10], end=[35, 20, 0], direction='z', fraction='+', value=10)

z [ 0.          3.88888889  7.77777778 11.66666667 15.55555556 19.44444444
 23.33333333 27.22222222 31.11111111 35.        ]
fraction 3


In [365]:
(38-13.5)/2

12.25

In [366]:
45.2-25.3

19.900000000000002

In [367]:
(45.2-25.3)/((38-13.5)/2)

1.6244897959183675

In [375]:
film = np.zeros((20, 20, 40))

paraStart = np.array([20, 10, 0])
paraEnd = np.array([20, 10, 20])

start = np.array([0, 15, 0])
end = np.array([0, 0, 37])

slitZ = np.linspace(end[2],  end[2] - paraEnd[2], paraEnd[0]-end[0])
slitY = np.linspace(start[1],  start[1] - paraStart[1], paraStart[0]-start[0])

print('slitZ',slitZ)
print('slitY',slitY)

for i in range(slitZ.shape[0]):
    film = slide3D_fractionZ(film=film, start=np.array([i, int(slitY[i]), 0]), end=np.array([i, 0, int(slitZ[i])]), direction='y', fraction='+', value=10)
# film = slide3D_fractionZ(film=film, start=[1, 10, 0], end=[20, 0, 35],paraStart=0, paraEnd=0, direction='y', fraction='+', value=10)
# film = slide3D_fractionZ(film=film, start=[2, 9, 0], end=[20, 0, 33],paraStart=0, paraEnd=0, direction='y', fraction='+', value=10)
# film = slide3D_fractionZ(film=film, start=[3, 8, 0], end=[20, 0, 31],paraStart=0, paraEnd=0, direction='y', fraction='+', value=10)
# film = slide3D_fractionZ(film=film, start=[4, 7, 0], end=[20, 0, 29],paraStart=0, paraEnd=0, direction='y', fraction='+', value=10)
# film = slide3D_fractionZ(film=film, start=[5, 6, 0], end=[20, 0, 27],paraStart=0, paraEnd=0, direction='y', fraction='+', value=10)
# film = slide3D_fractionZ(film=film, start=[6, 5, 0], end=[20, 0, 25],paraStart=0, paraEnd=0, direction='y', fraction='+', value=10)

slitZ [37.         35.94736842 34.89473684 33.84210526 32.78947368 31.73684211
 30.68421053 29.63157895 28.57894737 27.52631579 26.47368421 25.42105263
 24.36842105 23.31578947 22.26315789 21.21052632 20.15789474 19.10526316
 18.05263158 17.        ]
slitY [15.         14.47368421 13.94736842 13.42105263 12.89473684 12.36842105
 11.84210526 11.31578947 10.78947368 10.26315789  9.73684211  9.21052632
  8.68421053  8.15789474  7.63157895  7.10526316  6.57894737  6.05263158
  5.52631579  5.        ]
y [ 0.          2.64285714  5.28571429  7.92857143 10.57142857 13.21428571
 15.85714286 18.5        21.14285714 23.78571429 26.42857143 29.07142857
 31.71428571 34.35714286 37.        ]
fraction 2
y [ 0.          2.69230769  5.38461538  8.07692308 10.76923077 13.46153846
 16.15384615 18.84615385 21.53846154 24.23076923 26.92307692 29.61538462
 32.30769231 35.        ]
fraction 2
y [ 0.          2.83333333  5.66666667  8.5        11.33333333 14.16666667
 17.         19.83333333 22.66666667 25.5

In [376]:
depo1 = torch.Tensor(np.logical_and(film[:, :, :] !=0, film[:, :, :] < 10)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(film[:, :, :]==10).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show_grid()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:53499/index.html?ui=P_0x1d5125f95b0_103&reconnect=auto" class="pyv…

In [377]:
film[5, 2, :]

array([10.        , 10.        , 10.        , 10.        , 10.        ,
       10.        , 10.        , 10.        , 10.        , 10.        ,
       10.        , 10.        , 10.        , 10.        , 10.        ,
       10.        , 10.        , 10.        , 10.        , 10.        ,
       10.        , 10.        , 10.        , 10.        , 10.        ,
       10.        , 10.        , 10.        ,  0.66666667,  0.33333333,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ,
        0.        ,  0.        ,  0.        ,  0.        ,  0.        ])

In [284]:
(19-8.1)/(108-60.7869)

0.23086812770184548

In [285]:
(108-60.7869)/108

0.4371583333333333

In [289]:
(273-153)/273

0.43956043956043955

In [293]:
(48-20)/(273-153)

0.23333333333333334

In [302]:
16.2/45.2

0.35840707964601765

In [307]:
41/115

0.3565217391304348

In [308]:
120-41

79

In [309]:
8.1/5.5

1.4727272727272727

In [315]:
20/13

1.5384615384615385

In [378]:
19.0657/45.2

0.4218075221238938

In [448]:
(115-66)/115

0.4260869565217391

In [387]:
(45.2-25.1117)/45.2

0.4444314159292036

In [391]:
(115-64)/115

0.4434782608695652

In [475]:
115-64

51

In [392]:
(19 - 6.3692)/19

0.6647789473684211

In [402]:
(50 - 17)/50

0.66

In [403]:
50 - 17

33

In [404]:
(19 - 2.8934)/19

0.8477157894736842

In [409]:
(50 - 8)/50

0.84

In [411]:
(108 - 17.0446)/108

0.8421796296296296

In [419]:
(273 - 43)/273

0.8424908424908425

In [421]:
275 - 43

232

In [425]:
50-33

17

In [476]:
(45.2 - 19.1)/45.2

0.577433628318584

In [480]:
(115 - 49)/115

0.5739130434782609

In [481]:
115 - 49

66

In [482]:
(45.2 - 17.2)/45.2

0.6194690265486726

In [490]:
(115 - 44)/115

0.6173913043478261

In [491]:
115 - 44

71

In [492]:
(19-2.592)/19

0.8635789473684211

In [517]:
(50 - 7)/50

0.86

In [496]:
(19-1.742)/19

0.9083157894736842

In [519]:
(50 - 5)/50

0.9

In [554]:
film = np.zeros((277, 100, 120))

bottom = 5
film[:, :, 0:bottom] = 10 # bottom

side_wall = 2
film[:, :side_wall, :] = 10 # left
film[:, -side_wall:, :] = 10 # right


# #dfsdfsdfasdf
# height = 80
# left_side = 75
# right_side = 75

film = slide2D_fractionZ(film=film, start=[2, 67, 5], end=[88, 98, 56], direction='y', fraction='+', value=10)
film = slide2D_fractionZ(film=film, start=[2, 32, 5], end=[88, 1, 56], direction='y', fraction='+', value=10)

film = slide2D_fractionX(film=film, start=[275, 29, 0], end=[153, 1, 120], direction='y', fraction='-', value=10)
film = slide2D_fractionX(film=film, start=[275, 70, 0], end=[153, 98, 120], direction='y', fraction='-', value=10)

film = slide2D_fractionX(film=film, start=[275, 39, 0], end=[88, 1, 79], direction='y', fraction='-', value=10)
film = slide2D_fractionX(film=film, start=[275, 60, 0], end=[88, 98, 79], direction='y', fraction='-', value=10)


film = slide2D_fractionX(film=film, start=[275, 2, 4], end=[88, 98, 66], direction='z', fraction='-', value=10)

#----------left-----------
# -------- 3D slide------------
paraStart = np.array([232, 42, 51])
paraEnd = np.array([232, 17, 76])

start = np.array([88, 32, 5])
end = np.array([88, 1, 56])

slideZend = np.linspace(end[2], paraEnd[2], paraEnd[0]-end[0])
slideYstart = np.linspace(start[1], paraStart[1], paraStart[0]-start[0])

slideZstart = np.linspace(start[2], paraStart[2], paraStart[0]-start[0])
slideYend = np.linspace(end[1], paraEnd[1], paraEnd[0]-end[0])

print('slideZend',slideZend)
print('slideYstart',slideYstart)

print('slideZstart',slideZstart)
print('slideYend',slideYend)

for i in range(slideZend.shape[0]):
    film = slide3D_fractionZ(film=film, start=np.array([start[0]+i, int(slideYstart[i]), int(slideZstart[i])]), end=np.array([start[0]+i, int(slideYend[i]), int(slideZend[i])]), direction='y', fraction='+', value=10)

# -------- 3D slide------------

# -------- 3D slide------------
paraStart = np.array([275, 44, 66])
paraEnd = np.array([275, 35, 73])

start = np.array([232, 42, 51])
end = np.array([232, 17, 76])

slideZend = np.linspace(end[2], paraEnd[2], paraEnd[0]-end[0])
slideYstart = np.linspace(start[1], paraStart[1], paraStart[0]-start[0])

slideZstart = np.linspace(start[2], paraStart[2], paraStart[0]-start[0])
slideYend = np.linspace(end[1], paraEnd[1], paraEnd[0]-end[0])

print('slideZend',slideZend)
print('slideYstart',slideYstart)

print('slideZstart',slideZstart)
print('slideYend',slideYend)

for i in range(slideZend.shape[0]):
    film = slide3D_fractionZ(film=film, start=np.array([start[0]+i, int(slideYstart[i]), int(slideZstart[i])]), end=np.array([start[0]+i, int(slideYend[i]), int(slideZend[i])]), direction='y', fraction='+', value=10)

# -------- 3D slide------------


# --------right---------
# -------- 3D slide------------
paraStart = np.array([232, 58, 51])
paraEnd = np.array([232, 82, 76])

start = np.array([88, 67, 5])
end = np.array([88, 98, 56])

slideZend = np.linspace(end[2], paraEnd[2], paraEnd[0]-end[0])
slideYstart = np.linspace(start[1], paraStart[1], paraStart[0]-start[0])

slideZstart = np.linspace(start[2], paraStart[2], paraStart[0]-start[0])
slideYend = np.linspace(end[1], paraEnd[1], paraEnd[0]-end[0])

print('slideZend',slideZend)
print('slideYstart',slideYstart)

print('slideZstart',slideZstart)
print('slideYend',slideYend)

for i in range(slideZend.shape[0]):
    film = slide3D_fractionZ(film=film, start=np.array([start[0]+i, int(slideYstart[i]), int(slideZstart[i])]), end=np.array([start[0]+i, int(slideYend[i]), int(slideZend[i])]), direction='y', fraction='+', value=10)

# -------- 3D slide------------

# -------- 3D slide------------
paraStart = np.array([275, 56, 66])
paraEnd = np.array([275, 65, 73])

start = np.array([232, 58, 51])
end = np.array([232, 82, 76])

# paraStart = np.array([275, 44, 66])
# paraEnd = np.array([275, 35, 73])

# start = np.array([232, 42, 51])
# end = np.array([232, 17, 76])

slideZend = np.linspace(end[2], paraEnd[2], paraEnd[0]-end[0])
slideYstart = np.linspace(start[1], paraStart[1], paraStart[0]-start[0])

slideZstart = np.linspace(start[2], paraStart[2], paraStart[0]-start[0])
slideYend = np.linspace(end[1], paraEnd[1], paraEnd[0]-end[0])

print('slideZend',slideZend)
print('slideYstart',slideYstart)

print('slideZstart',slideZstart)
print('slideYend',slideYend)

for i in range(slideZend.shape[0]):
    film = slide3D_fractionZ(film=film, start=np.array([start[0]+i, int(slideYstart[i]), int(slideZstart[i])]), end=np.array([start[0]+i, int(slideYend[i]), int(slideZend[i])]), direction='y', fraction='+', value=10)

# -------- 3D slide------------



film[:side_wall, :, :] = 10 # back
film[-side_wall:, :, :] = 10 # front

y [ 0.   1.7  3.4  5.1  6.8  8.5 10.2 11.9 13.6 15.3 17.  18.7 20.4 22.1
 23.8 25.5 27.2 28.9 30.6 32.3 34.  35.7 37.4 39.1 40.8 42.5 44.2 45.9
 47.6 49.3 51. ]
fraction 1
y [ 0.   1.7  3.4  5.1  6.8  8.5 10.2 11.9 13.6 15.3 17.  18.7 20.4 22.1
 23.8 25.5 27.2 28.9 30.6 32.3 34.  35.7 37.4 39.1 40.8 42.5 44.2 45.9
 47.6 49.3 51. ]
fraction 1
y [  0.           4.51851852   9.03703704  13.55555556  18.07407407
  22.59259259  27.11111111  31.62962963  36.14814815  40.66666667
  45.18518519  49.7037037   54.22222222  58.74074074  63.25925926
  67.77777778  72.2962963   76.81481481  81.33333333  85.85185185
  90.37037037  94.88888889  99.40740741 103.92592593 108.44444444
 112.96296296 117.48148148 122.        ]
fraction 4
y [  0.           4.51851852   9.03703704  13.55555556  18.07407407
  22.59259259  27.11111111  31.62962963  36.14814815  40.66666667
  45.18518519  49.7037037   54.22222222  58.74074074  63.25925926
  67.77777778  72.2962963   76.81481481  81.33333333  85.85185185
  90.3

In [555]:
depo1 = torch.Tensor(np.logical_and(film[:, :, :] !=0, film[:, :, :] < 10)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(film[:, :, :]==10).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show_grid()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:53499/index.html?ui=P_0x1ccd4a7d670_150&reconnect=auto" class="pyv…

In [551]:
depo1 = torch.Tensor(np.logical_and(film[:, :50, :] !=0, film[:, :50, :] < 10)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(film[:, :50, :]==10).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)

p = pv.Plotter()
p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show_grid()
p.show()

c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\utilities\points.py:55: UserWarning: Points is not a float type. This can cause issues when transforming or applying filters. Casting to ``np.float32``. Disable this by passing ``force_float=False``.
  warnings.warn(
c:\Users\urcs049\AppData\Local\anaconda3\Lib\site-packages\pyvista\core\filters\data_set.py:2386: UserWarning: No vector-like data to use for orient. orient will be set to False.
  warnings.warn("No vector-like data to use for orient. orient will be set to False.")


Widget(value='<iframe src="http://localhost:53499/index.html?ui=P_0x1ccd4a7f050_148&reconnect=auto" class="pyv…

In [568]:
2.462/45.2

0.05446902654867257

In [570]:
6/115

0.05217391304347826

In [577]:
17.0446/108

0.15782037037037036

In [580]:
43/272

0.15808823529411764

In [582]:
(27.9233+35.52)/2/108

0.2937189814814815

In [585]:
80/272

0.29411764705882354

In [586]:
(48.4007+57.9357)/2/108

0.49229814814814815

In [596]:
134/272

0.49264705882352944

In [597]:
(58.4706+67.0673)/2/108

0.5811939814814815

In [600]:
158/272

0.5808823529411765

In [602]:
(67.6362+78.5445)/2/108

0.6767625

In [607]:
184/272

0.6764705882352942

In [608]:
(95.5445+78.5445)/2/108

0.8059675925925925

In [610]:
219/272

0.8051470588235294

In [613]:
(82.0445)/108

0.7596712962962963

In [619]:
207/272

0.7610294117647058

In [622]:
(95.5445 - 5) /108

0.838375

In [626]:
228/272

0.8382352941176471

In [628]:
9.5/38

0.25

In [632]:
25/100

0.25

In [634]:
0.25/3*2

0.16666666666666666

In [636]:
nozzle_1 = np.array([277-43-2, 50, 120-6])
nozzle_2 = np.array([277-80-2, 50, 120-6])
nozzle_3 = np.array([277-134-2, 50, 120-6])
nozzle_4 = np.array([277-158-2, 50, 120-6])

nozzle_5 = np.array([277-184-2, 50, 120-6])
nozzle_8 = np.array([277-184-2, 50-17, 120-6])
nozzle_11 = np.array([277-184-2, 50+17, 120-6])

nozzle_6 = np.array([277-207-2, 50, 120-6])
nozzle_9 = np.array([277-207-2, 50-17, 120-6])
nozzle_12 = np.array([277-207-2, 50+17, 120-6])

nozzle_7 = np.array([277-228-2, 50, 120-6])
nozzle_10 = np.array([277-228-2, 50-17, 120-6])
nozzle_13 = np.array([277-228-2, 50+17, 120-6])


In [637]:
depo1 = torch.Tensor(np.logical_and(film[:, :, :] !=0, film[:, :, :] < 10)).to_sparse()
depo1 = depo1.indices().numpy().T

substrute = torch.Tensor(film[:, :, :]==10).to_sparse()
substrute = substrute.indices().numpy().T
depomesh = pv.PolyData(depo1)
depomesh["radius"] = np.ones(depo1.shape[0])*0.5
geom = pv.Box()

submesh = pv.PolyData(substrute)
submesh["radius"] = np.ones(substrute.shape[0])*0.5

# Progress bar is a new feature on master branch
depoglyphed = depomesh.glyph(scale="radius", geom=geom) # progress_bar=True)
subglyphed = submesh.glyph(scale="radius", geom=geom) # progress_bar=True)


sphere1 = pv.Sphere(radius=4, center=nozzle_1)
sphere2 = pv.Sphere(radius=4, center=nozzle_2)
sphere3 = pv.Sphere(radius=4, center=nozzle_3)
sphere4 = pv.Sphere(radius=4, center=nozzle_4)
sphere5 = pv.Sphere(radius=4, center=nozzle_5)
sphere6 = pv.Sphere(radius=4, center=nozzle_6)
sphere7 = pv.Sphere(radius=4, center=nozzle_7)
sphere8 = pv.Sphere(radius=4, center=nozzle_8)
sphere9 = pv.Sphere(radius=4, center=nozzle_9)
sphere10 = pv.Sphere(radius=4, center=nozzle_10)
sphere11 = pv.Sphere(radius=4, center=nozzle_11)
sphere12 = pv.Sphere(radius=4, center=nozzle_12)
sphere13 = pv.Sphere(radius=4, center=nozzle_13)

p = pv.Plotter()

p.add_mesh(sphere1, show_edges=True, opacity=0.5, color="w")
p.add_mesh(sphere2, show_edges=True, opacity=0.5, color="w")
p.add_mesh(sphere3, show_edges=True, opacity=0.5, color="w")
p.add_mesh(sphere4, show_edges=True, opacity=0.5, color="w")
p.add_mesh(sphere5, show_edges=True, opacity=0.5, color="w")
p.add_mesh(sphere6, show_edges=True, opacity=0.5, color="w")
p.add_mesh(sphere7, show_edges=True, opacity=0.5, color="w")
p.add_mesh(sphere8, show_edges=True, opacity=0.5, color="w")
p.add_mesh(sphere9, show_edges=True, opacity=0.5, color="w")
p.add_mesh(sphere10, show_edges=True, opacity=0.5, color="w")
p.add_mesh(sphere11, show_edges=True, opacity=0.5, color="w")
p.add_mesh(sphere12, show_edges=True, opacity=0.5, color="w")
p.add_mesh(sphere13, show_edges=True, opacity=0.5, color="w")

p.add_mesh(depoglyphed, color='cyan')
p.add_mesh(subglyphed, color='dimgray')
p.enable_eye_dome_lighting()
p.show_grid()
p.show()

Widget(value='<iframe src="http://localhost:53499/index.html?ui=P_0x1cd4e3103e0_151&reconnect=auto" class="pyv…

In [638]:
np.save('ID445_boat', film)

In [ ]:
import DSMC_depoSimulator_generateWithtime_nozzle

In [66]:
logname = 'DSMCTS60_trenchDepo_0827'
test = DSMC_depoSimulator_generateWithtime_nozzle.depo(mirror=True,collision=False,velNormalize=True,pressure_pa=0.2, temperature=300, chamberSize=film.shape, DXsec=DXsec_Al,
                               param = [1.6, -0.7], TS = 0.3, N = int(1e6), 
                               sub_xy=[0,0], film=film, n=1, cellSize=film.shape, 
                               celllength=1e-5, kdtreeN=5, tstep=1e-5, thickness=110,substrateTop=80,posGeneratorType='gen1', logname=logname)

deposit = test.depo_position_increase_cosVel(125,  int(1e6), 2e-2, 0.3, 5)

d:\ysy\etching\Collision.py:31: RuntimeWarning: divide by zero encountered in divide
  diVr = d_refi * np.sqrt(((kb*Tref)/(eVr*self.q))**(wi-1/2)*gamma(5/2 - wi))
running:  36%|███████████████████                                  | 36.0/100 [07:31<13:22, 12.5s/B]

depo finish


In [68]:
np.save('cvd_void_subDepo_thk110', deposit[0])